# Network Flow Models
## Introduction to Linear Programming

Computational environments provide organizations with the ability to implement solutions to complex time, in real time.  

[Scikit Learn, Pulp, CPLEX, and Gurobi](https://medium.com/opex-analytics/optimization-modeling-in-python-pulp-gurobi-and-cplex-83a62129807a) are  Python packages which provide capabilities for Linear programming and optimization. 


In [ ]:
!pip install pulp

     |████████████████████████████████| 40.6MB 110kB/s 


In [ ]:
#Import some required packages. 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

This is adopted from the Pulp Examples [https://github.com/coin-or/pulp/blob/master/examples/BeerDistributionProblem.py](https://github.com/coin-or/pulp/blob/master/examples/BeerDistributionProblem.py) 

Copyright (c) 2002-2005, Jean-Sebastien Roy (js@jeannot.org)
Modifications Copyright (c) 2007- Stuart Anthony Mitchell (s.mitchell@auckland.ac.nz)

Permission is hereby granted, free of charge, to any person obtaining a
copy of this software and associated documentation files (the
"Software"), to deal in the Software without restriction, including
without limitation the rights to use, copy, modify, merge, publish,
distribute, sublicense, and/or sell copies of the Software, and to
permit persons to whom the Software is furnished to do so, subject to
the following conditions:

The above copyright notice and this permission notice shall be included
in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS
OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY
CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT,
TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
https://github.com/coin-or/pulp/blob/master/LICENSE


The problem motivation is found here:
https://www.coin-or.org/PuLP/CaseStudies/a_transportation_problem.html

In [ ]:
from pulp import *

# Creates a list of all the supply nodes
Warehouses = ["A", "B"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 1000,
          "B": 4000}

# Creates a list of all demand nodes
Bars = ["1", "2", "3", "4", "5"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {"1":500,
          "2":900,
          "3":1800,
          "4":200,
          "5":700,}

# Creates a list of costs of each transportation path
costs = [   #Bars
         #1 2 3 4 5
         [2,4,5,2,1],#A   Warehouses
         [3,1,3,2,3] #B
         ]



In [ ]:
# The cost data is made into a dictionary
costs = makeDict([Warehouses,Bars],costs,0)

In [ ]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("BeerDistributionProblem",LpMinimize)

In [ ]:
# Creates a list of tuples containing all the possible routes for transport
Routes = [(w,b) for w in Warehouses for b in Bars]

# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route",(Warehouses,Bars),0,None,LpInteger)

In [ ]:
# The objective function is added to 'prob' first
prob += lpSum([vars[w][b]*costs[w][b] for (w,b) in Routes]), "Sum_of_Transporting_Costs"

# The supply maximum constraints are added to prob for each supply node (warehouse)
for w in Warehouses:
    prob += lpSum([vars[w][b] for b in Bars])<=supply[w], "Sum_of_Products_out_of_Warehouse_%s"%w

# The demand minimum constraints are added to prob for each demand node (bar)
for b in Bars:
    prob += lpSum([vars[w][b] for w in Warehouses])>=demand[b], "Sum_of_Products_into_Bar%s"%b

## Solve

We now solve the system of equations with the solve command. 

In [ ]:
#Solve the program
prob.solve()


1

## Check the Status

Here are 5 status codes:
* **Not Solved**: Status prior to solving the problem.
* **Optimal**: An optimal solution has been found.
* **Infeasible**: There are no feasible solutions (e.g. if you set the constraints x <= 1 and x >=2).
* **Unbounded**: The constraints are not bounded, maximising the solution will tend towards infinity (e.g. if the only constraint was x >= 3).
* **Undefined**: The optimal solution may exist but may not have been found.

In [ ]:
pl.LpStatus[prob.status]

'Optimal'

In [ ]:
for variable in prob.variables():
    print(variable.name," = ", variable.varValue)

Route_A_1  =  300.0
Route_A_2  =  0.0
Route_A_3  =  0.0
Route_A_4  =  0.0
Route_A_5  =  700.0
Route_B_1  =  200.0
Route_B_2  =  900.0
Route_B_3  =  1800.0
Route_B_4  =  200.0
Route_B_5  =  0.0


In [ ]:
# The optimised objective function value is printed to the screen    
print("Total Cost of Transportation = ", value(prob.objective))

Total Cost of Transportation =  8600.0


## Sensitivity Analysis

In [21]:
for name, c in prob.constraints.items():
    print (name, ":", c, "\t", c.pi, "\t\t", c.slack)

Sum_of_Products_out_of_Warehouse_A : Route_A_1 + Route_A_2 + Route_A_3 + Route_A_4 + Route_A_5 <= 1000 	 0.0 		 -0.0
Sum_of_Products_out_of_Warehouse_B : Route_B_1 + Route_B_2 + Route_B_3 + Route_B_4 + Route_B_5 <= 4000 	 0.0 		 900.0
Sum_of_Products_into_Bar1 : Route_A_1 + Route_B_1 >= 500 	 0.0 		 -0.0
Sum_of_Products_into_Bar2 : Route_A_2 + Route_B_2 >= 900 	 0.0 		 -0.0
Sum_of_Products_into_Bar3 : Route_A_3 + Route_B_3 >= 1800 	 0.0 		 -0.0
Sum_of_Products_into_Bar4 : Route_A_4 + Route_B_4 >= 200 	 0.0 		 -0.0
Sum_of_Products_into_Bar5 : Route_A_5 + Route_B_5 >= 700 	 0.0 		 -0.0
